# Entrenamiento Local de Agentes Pokémon
Este cuaderno coordina el entrenamiento local de los agentes especializados y del agente híbrido usando las utilidades de `advanced_agents`. Cada sección describe qué configura o ejecuta para que puedas seguir el flujo sin consultar otros archivos. Para más detalles sobre los scripts equivalentes por lotes revisa `README_LOCAL_TRAINING.md`.

In [1]:
import sys
import os

# FIX: Resolver conflicto de OpenMP (Error #15) que causa crash del kernel
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import json
import shutil
import types
import importlib
from gymnasium import spaces

# Configuración de rutas locales
project_path = os.getcwd()
if project_path not in sys.path:
    sys.path.append(project_path)

baselines_path = os.path.join(project_path, 'baselines')
if baselines_path not in sys.path:
    sys.path.append(baselines_path)

print(f"Directorio de trabajo: {project_path}")

Directorio de trabajo: c:\Users\javi1\Documents\repos_git\TEL351-PokemonRed


## 1. Configuración de entorno
Inicializa rutas y variables de entorno necesarias para que PyBoy y Stable-Baselines3 funcionen sin conflictos (por ejemplo, se habilita `KMP_DUPLICATE_LIB_OK` para evitar errores de OpenMP).

## 1.1 Optimización con Numba (Opcional)
Para acelerar los cálculos de recompensa (especialmente el cálculo de percentiles en el historial de pérdidas), se recomienda instalar `numba`. Si no está instalado, el código usará una versión estándar de Python más lenta.

In [3]:
try:
    import numba
    print(f"Numba instalado: {numba.__version__}")
except ImportError:
    print("Numba no detectado. Instalando...")
    !pip install numba
    print("Instalación completada. Por favor reinicia el kernel si es necesario.")

# Verificar e instalar dependencias para la barra de progreso
try:
    import tqdm
    import rich
    import ipywidgets
    print(f"tqdm, rich e ipywidgets disponibles (barra de progreso activada)")
except ImportError:
    print("Instalando dependencias para habilitar la barra de progreso...")
    !pip install tqdm rich ipywidgets
    print("Instalación completada.")

Numba instalado: 0.62.1
tqdm, rich e ipywidgets disponibles (barra de progreso activada)


In [ ]:
try:
    import torch
    print(f"PyTorch GPU disponible: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"   Dispositivo: {torch.cuda.get_device_name(0)}")
    else:
        print("PyTorch está usando CPU. El entrenamiento será lento.")
except OSError as e:
    print(f"ERROR CRÍTICO DETECTADO: {e}")
    if "126" in str(e) or "caffe2_nvrtc.dll" in str(e):
        print("\n" + "="*60)
        print("   ¡NO TE PREOCUPES! ESTE ERROR ES ESPERADO SI LA VERSIÓN FALLA")
        print("   SOLUCIÓN: Ejecuta la SIGUIENTE CELDA para reparar PyTorch.")
        print("="*60 + "\n")
    else:
        raise e

PyTorch GPU disponible: True
   Dispositivo: NVIDIA GeForce RTX 3050


## 1.2 Solución de Problemas de GPU
Si la celda anterior indica que **PyTorch está usando CPU**, es probable que tengas instalada una versión incorrecta de PyTorch o que falten los drivers de CUDA.
Para arreglarlo en tu **RTX 3050**, ejecuta la siguiente celda para reinstalar una versión estable de PyTorch con soporte CUDA 12.4 (compatible con tus drivers actuales).
**Nota:** Después de la instalación, deberás reiniciar el kernel del notebook (Botón "Restart" en la barra superior).

In [5]:
# VERIFICACIÓN DE INSTALACIÓN
# Si esta celda falla con "WinError 126" o "ModuleNotFoundError", 
# SIGNIFICA QUE LA INSTALACIÓN ANTERIOR FALLÓ POR BLOQUEO DE ARCHIVOS.
#
# SOLUCIÓN:
# 1. Cierra este notebook o Reinicia el Kernel (Botón Restart ↻ arriba).
# 2. Abre una TERMINAL en VS Code (Ctrl+Ñ).
# 3. Copia y pega los comandos que te dio el asistente para reinstalar Torch manualmente.
# 4. Vuelve aquí y ejecuta esta celda.

import torch
print(f"Versión de Torch: {torch.__version__}")
print(f"Versión de CUDA en Torch: {torch.version.cuda}")
print(f"¿CUDA disponible?: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Detectada: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ GPU no detectada. Si tienes una RTX, reinstala Torch desde la terminal.")

Versión de Torch: 2.4.1+cu121
Versión de CUDA en Torch: 12.1
¿CUDA disponible?: True
GPU Detectada: NVIDIA GeForce RTX 3050


In [6]:
# --- RELOAD MODULES ---
def reload_modules():
    modules_to_reload = [
        'v2.red_gym_env_v2',
        'advanced_agents.features',
        'advanced_agents.wrappers',
        'advanced_agents.base',
        'advanced_agents.train_agents',
        'advanced_agents.combat_apex_agent',
        'advanced_agents.puzzle_speed_agent',
        'advanced_agents.hybrid_sage_agent',
        'advanced_agents.transition_models'
    ]
    for mod_name in modules_to_reload:
        if mod_name in sys.modules:
            try:
                importlib.reload(sys.modules[mod_name])
                print(f"Recargado: {mod_name}")
            except Exception as e:
                print(f"No se pudo recargar {mod_name}: {e}")

reload_modules()

## 2. Recarga de módulos
Permite refrescar los módulos clave de `advanced_agents` y del entorno RedGym cada vez que hagas cambios en el código fuente sin tener que reiniciar el kernel. Ejecuta esta celda si modificas archivos Python relacionados.

In [7]:
# Copiar events.json si es necesario
events_source = os.path.join(project_path, 'baselines', 'events.json')
events_dest = os.path.join(project_path, 'events.json')
if os.path.exists(events_source) and not os.path.exists(events_dest):
    shutil.copy(events_source, events_dest)
    print(f"Copiado events.json a {events_dest}")

## 3. Sincronización de `events.json`
Garantiza que el archivo de eventos requerido por PyBoy esté disponible en la raíz del proyecto copiándolo desde `baselines/events.json` cuando falta.

## 4. Utilidades de entrenamiento
Define el registro de agentes, valida que existan los archivos `.state`, construye las configuraciones de entorno y expone `train_single_run`/`train_plan`, que son los puntos de entrada para disparar los entrenamientos desde las celdas siguientes.

In [8]:
import json
import shutil
import types
import importlib
from typing import Dict, Iterable, List, Optional
import os

from gymnasium import spaces

try:
    from advanced_agents.train_agents import _base_env_config
    from advanced_agents.combat_apex_agent import CombatApexAgent, CombatAgentConfig
    from advanced_agents.puzzle_speed_agent import PuzzleSpeedAgent, PuzzleAgentConfig
    from advanced_agents.hybrid_sage_agent import HybridSageAgent, HybridAgentConfig
except ImportError as e:
    print(f"⚠️ ERROR DE IMPORTACIÓN: {e}")
    raise e
except OSError as e:
    print(f"⚠️ ERROR CRÍTICO DE PYTORCH: {e}")
    if "126" in str(e) or "caffe2_nvrtc.dll" in str(e):
        print("\n" + "="*60)
        print("   ¡TU INSTALACIÓN DE PYTORCH ESTÁ ROTA!")
        print("   El kernel tiene archivos bloqueados o la versión es incompatible.")
        print("   ")
        print("   SOLUCIÓN DEFINITIVA:")
        print("   1. Abre la terminal (Ctrl+Ñ)")
        print("   2. Ejecuta: ./repair_torch.ps1")
        print("   3. Reinicia el Kernel (Botón Restart ↻)")
        print("="*60 + "\n")
    raise e

# --- Cargar escenarios ---
SCENARIO_PATH = os.path.join(project_path, 'gym_scenarios', 'scenarios.json')
with open(SCENARIO_PATH, 'r') as f:
    scenarios_data = json.load(f)

SCENARIOS: Dict[str, Dict] = {scenario['id']: scenario for scenario in scenarios_data['scenarios']}

AGENT_REGISTRY = {
    'combat': {
        'agent_cls': CombatApexAgent,
        'config_cls': CombatAgentConfig,
        'default_phase': 'battle'
    },
    'puzzle': {
        'agent_cls': PuzzleSpeedAgent,
        'config_cls': PuzzleAgentConfig,
        'default_phase': 'puzzle'
    },
    'hybrid': {
        'agent_cls': HybridSageAgent,
        'config_cls': HybridAgentConfig,
        'default_phase': 'battle'
    }
}

MODELS_DIR = os.path.join(project_path, 'models_local')
os.makedirs(MODELS_DIR, exist_ok=True)

def resolve_phase(scenario_id: str, phase_name: Optional[str]) -> Dict:
    scenario = SCENARIOS.get(scenario_id)
    if scenario is None:
        raise ValueError(f"Escenario {scenario_id} no encontrado en {SCENARIO_PATH}")
    target_phase = phase_name or AGENT_REGISTRY['combat']['default_phase']
    selected_phase = next((p for p in scenario['phases'] if p['name'] == target_phase), None)
    if selected_phase is None:
        raise ValueError(f"Fase {target_phase} no encontrada en el escenario {scenario_id}")
    return selected_phase

def ensure_state_file(state_file_path: str) -> str:
    abs_path = os.path.join(project_path, state_file_path) if not os.path.isabs(state_file_path) else state_file_path
    if not os.path.exists(abs_path):
        raise FileNotFoundError(
            f"No se encontró el archivo de estado requerido: {abs_path}. "
            "Genera los .state con generate_gym_states.py o ajusta la ruta."
        )
    return abs_path

def build_env_overrides(state_file_path: str, headless: bool) -> Dict:
    return {
        'init_state': state_file_path,
        'headless': headless,
        'save_video': False,
        'gb_path': os.path.join(project_path, 'PokemonRed.gb'),
        'session_path': os.path.join(project_path, 'sessions', f"local_{os.path.basename(state_file_path)}"),
        'render_mode': 'rgb_array' if headless else 'human',
        'fast_video': headless
    }

def _patch_callbacks(agent, additional_callbacks: Optional[List] = None):
    base_callbacks_method = agent.extra_callbacks

    def _patched_callbacks(self):
        callbacks = list(base_callbacks_method())
        if additional_callbacks:
            callbacks.extend(additional_callbacks)
        return callbacks

    agent.extra_callbacks = types.MethodType(_patched_callbacks, agent)

def train_single_run(
    agent_key: str,
    scenario_id: str,
    phase_name: str,
    total_timesteps: int = 200_000,
    headless: bool = False,
    additional_callbacks: Optional[List] = None
):
    registry_entry = AGENT_REGISTRY.get(agent_key)
    if registry_entry is None:
        raise ValueError(f"Agente desconocido: {agent_key}")

    phase = resolve_phase(scenario_id, phase_name)
    state_file_path = ensure_state_file(phase['state_file'])

    env_overrides = build_env_overrides(state_file_path, headless=headless)
    config = registry_entry['config_cls'](
        env_config=_base_env_config(env_overrides),
        total_timesteps=total_timesteps
    )

    agent = registry_entry['agent_cls'](config)

    env_for_check = agent.make_env()
    obs_space = getattr(env_for_check, 'observation_space', None)
    if isinstance(obs_space, spaces.Dict):
        print("Observación Dict detectada -> MultiInputPolicy")
        agent.policy_name = types.MethodType(lambda self: "MultiInputPolicy", agent)
    env_for_check.close()

    if additional_callbacks:
        _patch_callbacks(agent, additional_callbacks)

    print(
        f"\n=== Entrenando {agent_key.upper()} en {scenario_id} ({phase_name}) por {total_timesteps:,} pasos ===")
    runtime = agent.train()

    agent_dir = os.path.join(MODELS_DIR, agent_key)
    os.makedirs(agent_dir, exist_ok=True)
    model_path = os.path.join(agent_dir, f"{scenario_id}_{phase_name}.zip")
    runtime.model.save(model_path)
    print(f"Modelo guardado en {model_path}")

    return runtime

def train_plan(
    agent_key: str,
    plan: List[Dict],
    default_timesteps: int = 200_000,
    headless: bool = False,
    callback_factory: Optional[callable] = None
) -> Dict[tuple, object]:
    results = {}
    total_runs = len(plan)
    for run_idx, entry in enumerate(plan, start=1):
        scenario_id = entry['scenario']
        phase_name = entry.get('phase') or AGENT_REGISTRY[agent_key]['default_phase']
        run_timesteps = entry.get('timesteps', default_timesteps)
        callbacks = None
        if callback_factory is not None:
            callbacks = callback_factory(entry)
        print(f"\n>>> [{agent_key.upper()}] Ejecución {run_idx}/{total_runs}")
        runtime = train_single_run(
            agent_key=agent_key,
            scenario_id=scenario_id,
            phase_name=phase_name,
            total_timesteps=run_timesteps,
            headless=headless,
            additional_callbacks=callbacks
        )
        results[(scenario_id, phase_name)] = runtime
    return results

## 5. Planes de entrenamiento
Ajusta aquí qué escenarios, fases y pasos quieres cubrir para cada agente. Usa esto como checklist antes de lanzar ejecuciones largas; puedes sobreescribir timesteps por fila y alternar `headless` para ver la ventana del emulador.

### Configura planes de entrenamiento locales
Especifica los escenarios, fases y timesteps que quieres para cada agente. Puedes ejecutar cada bloque por separado y combinar headless=True/False según quieras ver la ventana del emulador.

In [9]:
# NOTA: Para pruebas rápidas con pocos pasos (ej. 200), considera reducir n_steps
# en la configuración del agente, ya que PPO hace rollouts completos de n_steps=1024 por defecto.
# Para entrenamiento real, usa valores como 40_000+ timesteps.

combat_plan_local = [
    {"scenario": "pewter_brock", "phase": "battle", "timesteps": 40_000},
    # {"scenario": "cerulean_misty", "phase": "battle", "timesteps": 50_000},
]

puzzle_plan_local = [
    {"scenario": "pewter_brock", "phase": "puzzle", "timesteps": 40_000},
    # {"scenario": "cerulean_misty", "phase": "puzzle", "timesteps": 50_000},
]

hybrid_plan_local = [
    {"scenario": "pewter_brock", "phase": "battle", "timesteps": 50_000},
    # {"scenario": "vermillion_lt_surge", "phase": "battle", "timesteps": 60_000},
]

DEFAULT_TIMESTEPS_LOCAL = 40_000
DEFAULT_HEADLESS_LOCAL = False  # Cambia a True si no necesitas la ventana SDL

## 6. Ejecutar plan de combate

**IMPORTANTE**: Si tu entrenamiento anterior mostró `value_loss > 1000` o `explained_variance < 0.1`, el modelo **no aprendió correctamente**. 

**Síntomas de entrenamiento fallido:**
- value_loss = 3200 (debería estar cerca de 0)
- explained_variance = 0.036 (debería ser >0.5)
- Reward constante en evaluación
- Episodios terminan en timeout sin progreso

**Solución**: La siguiente celda usa parámetros **estabilizados** automáticamente. Solo ejecútala para re-entrenar con configuración robusta.

In [ ]:
# ==================== ENTRENAMIENTO CON PARÁMETROS ESTABLES ====================
# Si tu entrenamiento anterior falló (value_loss alto), esta versión usa parámetros
# más conservadores que garantizan convergencia.
# =================================================================================

from advanced_agents.combat_apex_agent import CombatApexAgent, CombatAgentConfig

def train_combat_stable(scenario_id='pewter_brock', phase_name='battle', timesteps=40_000):
    """Entrena CombatApexAgent con parámetros estabilizados."""
    print(f"\n{'='*70}")
    print(f"   ENTRENAMIENTO ESTABLE - COMBAT APEX AGENT")
    print(f"   Escenario: {scenario_id} | Fase: {phase_name}")
    print(f"   Pasos: {timesteps:,}")
    print(f"   Parámetros: LR reducido, clipping conservador, gradientes limitados")
    print(f"{'='*70}\n")
    
    # Configurar entorno
    phase = resolve_phase(scenario_id, phase_name)
    state_file_path = ensure_state_file(phase['state_file'])
    env_overrides = build_env_overrides(state_file_path, headless=True)
    base_config = _base_env_config(env_overrides)
    
    # Configuración ESTABLE (parámetros ajustados para evitar divergencia)
    agent_config = CombatAgentConfig(
        env_config=base_config,
        total_timesteps=timesteps,
        learning_rate=1e-4,      # Más conservador que 2.5e-4
        n_steps=512,             # Actualizaciones más frecuentes
        batch_size=128,          # Batches más pequeños
        gamma=0.998,             # Menos influencia del futuro
        gae_lambda=0.95,
        clip_range=0.1,          # Clipping más estricto
        vf_coef=0.25,            # Menos peso a la función de valor
        ent_coef=0.01,           # Entropía para exploración
        device='cuda' if torch.cuda.is_available() else 'cpu'
    )
    
    # Crear agente
    agent = CombatApexAgent(agent_config)
    
    # Verificar espacio de observaciones
    env_check = agent.make_env()
    from gymnasium import spaces
    if isinstance(env_check.observation_space, spaces.Dict):
        print("Observación Dict detectada -> MultiInputPolicy")
        agent.policy_name = lambda: "MultiInputPolicy"
    env_check.close()
    
    # Entrenar
    print(f"\n🚀 Iniciando entrenamiento estable...")
    runtime = agent.train()
    
    # Guardar
    save_dir = os.path.join(MODELS_DIR, 'combat')
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f"{scenario_id}_{phase_name}_stable.zip")
    runtime.model.save(save_path)
    
    print(f"\nModelo ESTABLE guardado en: {save_path}")
    print(f"Revisa los logs - deberías ver:")
    print(f"   - value_loss < 100 (idealmente < 10)")
    print(f"   - explained_variance > 0.3 (mejorando hacia 0.7+)")
    print(f"   - approx_kl < 0.05")
    
    return save_path

# EJECUTAR ENTRENAMIENTO ESTABLE
combat_model_stable = train_combat_stable(
    scenario_id='pewter_brock',
    phase_name='battle', 
    timesteps=40_000
)

print(f"\n{'='*70}")
print(f"ENTRENAMIENTO COMPLETO")
print(f"{'='*70}")
print(f"Modelo guardado en: {combat_model_stable}")
print(f"\n🎮 Para probarlo:")
print(f"python run_combat_agent_interactive.py --scenario pewter_brock --phase battle")
print(f"(Renombra el archivo _stable.zip a .zip si es necesario)")
print(f"{'='*70}")


>>> [COMBAT] Ejecución 1/1
Observación Dict detectada -> MultiInputPolicy

=== Entrenando COMBAT en pewter_brock (battle) por 40,000 pasos ===


Output()

Using cuda device
Wrapping the env in a VecTransposeImage.


-----------------------------
| time/              |      |
|    fps             | 5    |
|    iterations      | 1    |
|    time_elapsed    | 191  |
|    total_timesteps | 1024 |
-----------------------------


------------------------------------------
| time/                   |              |
|    fps                  | 5            |
|    iterations           | 2            |
|    time_elapsed         | 405          |
|    total_timesteps      | 2048         |
| train/                  |              |
|    approx_kl            | 0.007673993  |
|    clip_fraction        | 0.195        |
|    clip_range           | 0.15         |
|    entropy_loss         | -1.94        |
|    explained_variance   | 0.0006688237 |
|    learning_rate        | 0.00025      |
|    loss                 | 6.58e+03     |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.00447     |
|    value_loss           | 3.3e+03      |
------------------------------------------


...

--------------------------------------------
| time/                   |                |
|    fps                  | 4              |
|    iterations           | 37             |
|    time_elapsed         | 7836           |
|    total_timesteps      | 37888          |
| train/                  |                |
|    approx_kl            | 0.0071684606   |
|    clip_fraction        | 0.14           |
|    clip_range           | 0.15           |
|    entropy_loss         | -1.84          |
|    explained_variance   | -1.1920929e-07 |
|    learning_rate        | 0.00025        |
|    loss                 | -0.0319        |
|    n_updates            | 360            |
|    policy_gradient_loss | -0.00321       |
|    value_loss           | 0.000568       |
--------------------------------------------


-----------------------------------------
| time/                   |             |
|    fps                  | 4           |
|    iterations           | 38          |
|    time_elapsed         | 8047        |
|    total_timesteps      | 38912       |
| train/                  |             |
|    approx_kl            | 0.013177071 |
|    clip_fraction        | 0.158       |
|    clip_range           | 0.15        |
|    entropy_loss         | -1.83       |
|    explained_variance   | 0.035197556 |
|    learning_rate        | 0.00025     |
|    loss                 | -0.00521    |
|    n_updates            | 370         |
|    policy_gradient_loss | -0.00362    |
|    value_loss           | 3.2e+03     |
-----------------------------------------


-------------------------------------------
| time/                   |               |
|    fps                  | 4             |
|    iterations           | 39            |
|    time_elapsed         | 8258          |
|    total_timesteps      | 39936         |
| train/                  |               |
|    approx_kl            | 0.009246286   |
|    clip_fraction        | 0.212         |
|    clip_range           | 0.15          |
|    entropy_loss         | -1.82         |
|    explained_variance   | 1.1920929e-07 |
|    learning_rate        | 0.00025       |
|    loss                 | -0.0392       |
|    n_updates            | 380           |
|    policy_gradient_loss | -0.00546      |
|    value_loss           | 0.00176       |
-------------------------------------------


-----------------------------------------
| time/                   |             |
|    fps                  | 4           |
|    iterations           | 40          |
|    time_elapsed         | 8455        |
|    total_timesteps      | 40960       |
| train/                  |             |
|    approx_kl            | 0.011068063 |
|    clip_fraction        | 0.229       |
|    clip_range           | 0.15        |
|    entropy_loss         | -1.82       |
|    explained_variance   | 0.036417663 |
|    learning_rate        | 0.00025     |
|    loss                 | -0.0234     |
|    n_updates            | 390         |
|    policy_gradient_loss | -0.00463    |
|    value_loss           | 3.2e+03     |
-----------------------------------------


Modelo guardado en c:\Users\javi1\Documents\repos_git\TEL351-PokemonRed\models_local\combat\pewter_brock_battle.zip


## 7. Ejecutar plan de puzzles
Corre el plan `puzzle_plan_local` usando `PuzzleSpeedAgent` y guarda salidas en `models_local/puzzle/`. Útil para medir tiempos de navegación y resolución de puzzles previos al combate.

In [10]:
puzzle_runs_local = train_plan(
    agent_key='puzzle',
    plan=puzzle_plan_local,
    default_timesteps=DEFAULT_TIMESTEPS_LOCAL,
    headless=DEFAULT_HEADLESS_LOCAL
)


>>> [PUZZLE] Ejecución 1/1
Observación Dict detectada -> MultiInputPolicy

=== Entrenando PUZZLE en pewter_brock (puzzle) por 200 pasos ===
Using cuda device
Wrapping the env in a VecTransposeImage.


AttributeError: 'RedGymEnv' object has no attribute 'seen_coords'

## 8. Ejecutar plan híbrido
Activa `HybridSageAgent` sobre los escenarios definidos en `hybrid_plan_local`, mezclando comportamientos de combate y navegación y almacenando resultados en `models_local/hybrid/`.

In [ ]:
hybrid_runs_local = train_plan(
    agent_key='hybrid',
    plan=hybrid_plan_local,
    default_timesteps=DEFAULT_TIMESTEPS_LOCAL,
    headless=DEFAULT_HEADLESS_LOCAL
)

## 9. Guardado manual (opcional)
Fragmento de ejemplo para guardar un modelo entrenado con un nombre personalizado. Solo úsalo si traes a la sesión variables como `model`, `AGENT_TYPE`, `SCENARIO_ID` y `PHASE_NAME`; de lo contrario producirá errores.

In [20]:
# Guardar modelo
save_dir = "models_local"
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, f"{AGENT_TYPE}_{SCENARIO_ID}_{PHASE_NAME}")
model.save(save_path)
print(f"Modelo guardado en {save_path}")

NameError: name 'AGENT_TYPE' is not defined

## 10. Comparación con Baseline (PPO v2)

Esta sección permite comparar el desempeño de tus agentes entrenados (Combat, Puzzle, Hybrid) contra un baseline.

**IMPORTANTE - Limitaciones de RAM (16GB):**
- El modelo `poke_26214400.zip` (26M pasos) requiere >10GB solo para cargarlo
- **Alternativa recomendada**: Entrenar tu propio baseline ligero (40k-100k pasos) en lugar de usar el modelo pesado
- O simplemente evaluar solo tus modelos locales sin comparación (ver celda siguiente)

**Alternativa para comparar sin .zip pesado:**
Puedes usar `run_pretrained_interactive.py` como baseline ejecutándolo manualmente y registrando las métricas, pero esta sección automatiza la evaluación de **tus modelos** sin necesidad del baseline gigante.

In [ ]:
import pandas as pd
import numpy as np
from stable_baselines3 import PPO
from v2.red_gym_env_v2 import RedGymEnv

def load_baseline_model(path):
    if not os.path.exists(path):
        print(f"No se encontró el modelo baseline en: {path}")
        return None
    try:
        return PPO.load(path)
    except Exception as e:
        print(f"Error cargando baseline: {e}")
        return None

def evaluate_agent_model(model, env, num_episodes=1):
    """Ejecuta episodios de evaluación y retorna métricas promedio."""
    rewards = []
    steps = []
    
    for i in range(num_episodes):
        # Manejar diferentes formatos de reset()
        reset_result = env.reset()
        obs = reset_result[0] if isinstance(reset_result, tuple) else reset_result
        
        done = False
        truncated = False
        total_reward = 0
        step_count = 0
        max_steps = 5000  # Límite de pasos por episodio
        
        while not done and not truncated and step_count < max_steps:
            action, _ = model.predict(obs, deterministic=True)
            step_result = env.step(action)
            
            # Manejar diferentes formatos de step()
            if len(step_result) == 5:
                obs, reward, done, truncated, info = step_result
            elif len(step_result) == 4:
                obs, reward, done, info = step_result
                truncated = False
            else:
                raise ValueError(f"Formato inesperado de step(): {len(step_result)} valores")
            
            # Convertir reward a escalar
            reward_scalar = float(reward.item() if hasattr(reward, 'item') else reward)
            total_reward += reward_scalar
            step_count += 1
            
        rewards.append(total_reward)
        steps.append(step_count)
        
    return {
        'mean_reward': np.mean(rewards),
        'std_reward': np.std(rewards),
        'mean_steps': np.mean(steps)
    }

def run_comparison_lightweight(plans_dict, baseline_path=None, headless=True, skip_baseline=False):
    """
    Versión optimizada para RAM limitada (<=16GB con Windows ocupando 10GB).
    skip_baseline=True: Solo evalúa tus modelos locales (recomendado para 16GB RAM)
    """
    results = []
    
    # Cargar Baseline solo si se solicita y existe
    baseline_model = None
    if not skip_baseline and baseline_path:
        print(f"Intentando cargar modelo baseline desde: {baseline_path}")
        baseline_model = load_baseline_model(baseline_path)
        if not baseline_model:
            print("No se pudo cargar el baseline. Solo se evaluarán modelos locales.")
    else:
        print("Modo sin baseline activado (ahorra ~10GB RAM)")
    
    for agent_key, plan in plans_dict.items():
        for entry in plan:
            scenario_id = entry['scenario']
            phase_name = entry.get('phase') or AGENT_REGISTRY[agent_key]['default_phase']
            
            print(f"\n--- Evaluando {agent_key.upper()} en {scenario_id} ({phase_name}) ---")
            
            # 1. Preparar Configuración Común
            phase = resolve_phase(scenario_id, phase_name)
            state_file_path = ensure_state_file(phase['state_file'])
            env_overrides = build_env_overrides(state_file_path, headless=headless)
            base_config = _base_env_config(env_overrides)
            
            # ---------------------------------------------------------
            # 2. Evaluar Agente Local (con su propio wrapper/env)
            # ---------------------------------------------------------
            registry_entry = AGENT_REGISTRY[agent_key]
            agent_config = registry_entry['config_cls'](
                env_config=base_config,
                total_timesteps=1000 
            )
            local_agent_wrapper = registry_entry['agent_cls'](agent_config)
            
            local_model_path = os.path.join(MODELS_DIR, agent_key, f"{scenario_id}_{phase_name}.zip")
            
            if os.path.exists(local_model_path):
                print(f"Cargando modelo local: {local_model_path}")
                try:
                    env_local = local_agent_wrapper.make_env()
                    local_agent_wrapper.model = PPO.load(local_model_path)
                    
                    print(f"Ejecutando evaluación...")
                    metrics_local = evaluate_agent_model(local_agent_wrapper.model, env_local)
                    env_local.close()
                    
                    print(f"Local: Reward={metrics_local['mean_reward']:.2f}, Steps={metrics_local['mean_steps']:.0f}")
                    
                    results.append({
                        'Agent': agent_key.upper(),
                        'Scenario': scenario_id,
                        'Phase': phase_name,
                        'Model': 'Local (Specialized)',
                        'Reward': metrics_local['mean_reward'],
                        'Steps': metrics_local['mean_steps']
                    })
                    
                    # Liberar memoria
                    del local_agent_wrapper.model
                    del env_local
                    
                except Exception as e:
                    print(f"Error evaluando local: {e}")
                    import traceback
                    traceback.print_exc()
            else:
                print(f"No existe modelo local en {local_model_path}")

            # ---------------------------------------------------------
            # 3. Evaluar Baseline solo si está disponible
            # ---------------------------------------------------------
            if baseline_model:
                print("Evaluando Baseline...")
                try:
                    env_baseline = RedGymEnv(base_config)
                    metrics_baseline = evaluate_agent_model(baseline_model, env_baseline)
                    env_baseline.close()
                    
                    print(f"Baseline: Reward={metrics_baseline['mean_reward']:.2f}, Steps={metrics_baseline['mean_steps']:.0f}")
                    
                    results.append({
                        'Agent': agent_key.upper(),
                        'Scenario': scenario_id,
                        'Phase': phase_name,
                        'Model': 'Baseline (PPO v2)',
                        'Reward': metrics_baseline['mean_reward'],
                        'Steps': metrics_baseline['mean_steps']
                    })
                    
                    del env_baseline
                    
                except Exception as e:
                    print(f"Error evaluando baseline: {e}")

    return pd.DataFrame(results) if results else None

In [23]:
# ==================== CONFIGURACIÓN DE COMPARACIÓN ====================
# Para sistemas con 16GB RAM (con Windows usando ~10GB):
# skip_baseline=True: Solo evalúa tus modelos (ahorra ~10GB)
# skip_baseline=False: Intenta cargar el baseline (requiere >20GB RAM total)
# ========================================================================

BASELINE_MODEL_PATH = os.path.join(project_path, 'v2', 'runs', 'poke_26214400.zip')

comparison_plans = {
    'combat': combat_plan_local,
    # 'puzzle': puzzle_plan_local,   # Comenta para evaluar menos modelos
    # 'hybrid': hybrid_plan_local,   # Comenta para evaluar menos modelos
}

# IMPORTANTE: skip_baseline=True para ahorrar RAM
df_results = run_comparison_lightweight(
    comparison_plans, 
    baseline_path=BASELINE_MODEL_PATH, 
    headless=True,
    skip_baseline=True  # Cambia a False solo si tienes >24GB RAM
)

if df_results is not None and not df_results.empty:
    print("\n" + "="*60)
    print("           RESULTADOS DE EVALUACIÓN")
    print("="*60)
    print(df_results.to_string(index=False))
    print("="*60)
    
    # Guardar CSV
    csv_path = "evaluacion_modelos_locales.csv"
    df_results.to_csv(csv_path, index=False)
    print(f"\nResultados guardados en: {csv_path}")
else:
    print("\nNo se generaron resultados. Verifica que existan modelos entrenados.")

Modo sin baseline activado (ahorra ~10GB RAM)

--- Evaluando COMBAT en pewter_brock (battle) ---
Cargando modelo local: c:\Users\javi1\Documents\repos_git\TEL351-PokemonRed\models_local\combat\pewter_brock_battle.zip
Error evaluando local: too many values to unpack (expected 2)

No se generaron resultados. Verifica que existan modelos entrenados.


## 11. Entrenar Baseline Ligero (Opcional - Alternativa al modelo pesado)

Si quieres comparar tus agentes especializados con un baseline PPO genérico **sin usar el modelo gigante de 26M pasos**, puedes entrenar tu propio baseline ligero aquí. Este será un modelo estándar de `v2/red_gym_env_v2.py` entrenado con los **mismos 40k pasos** que tus agentes especializados para una comparación justa.

In [ ]:
from stable_baselines3 import PPO
from v2.red_gym_env_v2 import RedGymEnv

def train_lightweight_baseline(scenario_id='pewter_brock', phase_name='battle', timesteps=40_000):
    """
    Entrena un baseline PPO simple (sin wrappers especializados) para comparación justa.
    Usa el mismo número de pasos que tus agentes especializados.
    """
    print(f"\n{'='*60}")
    print(f"   ENTRENANDO BASELINE LIGERO (PPO Genérico)")
    print(f"   Escenario: {scenario_id} | Fase: {phase_name}")
    print(f"   Pasos: {timesteps:,}")
    print(f"{'='*60}\n")
    
    # Preparar configuración del entorno (igual que tus agentes)
    phase = resolve_phase(scenario_id, phase_name)
    state_file_path = ensure_state_file(phase['state_file'])
    env_overrides = build_env_overrides(state_file_path, headless=True)
    base_config = _base_env_config(env_overrides)
    
    # Crear entorno estándar (sin wrappers especializados)
    env = RedGymEnv(base_config)
    
    # Crear modelo PPO con configuración similar a tus agentes
    model = PPO(
        "CnnPolicy",  # Política estándar para imágenes
        env,
        learning_rate=2.5e-4,
        n_steps=1024,
        batch_size=256,
        gamma=0.999,
        verbose=1,
        device='cuda' if torch.cuda.is_available() else 'cpu'
    )
    
    # Entrenar
    try:
        import tqdm, rich
        model.learn(total_timesteps=timesteps, progress_bar=True)
    except ImportError:
        model.learn(total_timesteps=timesteps)
    
    # Guardar
    baseline_dir = os.path.join(MODELS_DIR, 'baseline_lightweight')
    os.makedirs(baseline_dir, exist_ok=True)
    baseline_path = os.path.join(baseline_dir, f"{scenario_id}_{phase_name}.zip")
    model.save(baseline_path)
    
    print(f"\nBaseline ligero guardado en: {baseline_path}")
    env.close()
    
    return baseline_path

# Entrenar baseline (descomenta para ejecutar)
# baseline_ligero_path = train_lightweight_baseline(
#     scenario_id='pewter_brock',
#     phase_name='battle',
#     timesteps=40_000  # Mismo número de pasos que tus agentes
# )

### Comparar con Baseline Ligero

Una vez entrenado el baseline ligero, puedes compararlo con tus agentes especializados usando esta celda:

In [ ]:
# Ruta al baseline ligero que acabas de entrenar
BASELINE_LIGERO_PATH = os.path.join(project_path, 'models_local', 'baseline_lightweight', 'pewter_brock_battle.zip')

# Comparar (solo si el baseline ligero existe)
if os.path.exists(BASELINE_LIGERO_PATH):
    print("Comparando con Baseline Ligero (entrenado con los mismos 40k pasos)")
    
    df_comparison = run_comparison_lightweight(
        {'combat': combat_plan_local},
        baseline_path=BASELINE_LIGERO_PATH,
        headless=True,
        skip_baseline=False  # Ahora SÍ cargamos el baseline (es ligero)
    )
    
    if df_comparison is not None and not df_comparison.empty:
        print("\n" + "="*70)
        print("     COMPARACIÓN: AGENTE ESPECIALIZADO vs BASELINE LIGERO")
        print("="*70)
        print(df_comparison.to_string(index=False))
        print("="*70)
        
        # Calcular mejora
        if len(df_comparison) == 2:
            reward_especializado = df_comparison[df_comparison['Model'].str.contains('Specialized')]['Reward'].values[0]
            reward_baseline = df_comparison[df_comparison['Model'].str.contains('Baseline')]['Reward'].values[0]
            mejora = ((reward_especializado - reward_baseline) / abs(reward_baseline)) * 100
            print(f"\nMejora del agente especializado: {mejora:+.1f}%")
        
        df_comparison.to_csv("comparacion_especializado_vs_baseline_ligero.csv", index=False)
else:
    print(f"Baseline ligero no encontrado en: {BASELINE_LIGERO_PATH}")
    print("Ejecuta primero la celda anterior para entrenar el baseline ligero.")

In [ ]:
# =================================================================================
# EVALUACIÓN DE AGENTE EN ESCENARIO DE GIMNASIO (HEADLESS + MÉTRICAS)
# =================================================================================

import json
import time
import sys
import numpy as np
from pathlib import Path
from stable_baselines3 import PPO
from gym_scenarios.gym_metrics import GymMetricsTracker

# Importar direcciones de memoria necesarias para la inyección
# (Copiadas de memory_addresses.py para asegurar disponibilidad)
PARTY_SIZE_ADDRESS = 0xD163
PARTY_ADDRESSES = [0xD164, 0xD165, 0xD166, 0xD167, 0xD168, 0xD169]
LEVELS_ADDRESSES = [0xD18C, 0xD1B8, 0xD1E4, 0xD210, 0xD23C, 0xD268]
HP_ADDRESSES = [0xD16C, 0xD198, 0xD1C4, 0xD1F0, 0xD21C, 0xD248]
MAX_HP_ADDRESSES = [0xD18D, 0xD1B9, 0xD1E5, 0xD211, 0xD23D, 0xD269]
MONEY_ADDRESS_1 = 0xD347
MONEY_ADDRESS_2 = 0xD348
MONEY_ADDRESS_3 = 0xD349
BADGE_COUNT_ADDRESS = 0xD356
BAG_ITEMS_START = 0xD31E
BAG_ITEM_COUNT = 0xD31D

def get_base_env(env):
    """Obtiene el entorno base (RedGymEnv) de un wrapper o VecEnv."""
    if hasattr(env, 'envs'): # DummyVecEnv
        env = env.envs[0]
    if hasattr(env, 'unwrapped'):
        return env.unwrapped
    return env

def inject_gym_config(env, config):
    """Inyecta la configuración del equipo e inventario en la memoria del emulador."""
    base_env = get_base_env(env)
    pyboy = base_env.pyboy
    
    def write_mem(addr, val):
        if hasattr(pyboy, "set_memory_value"):
            pyboy.set_memory_value(addr, val & 0xFF)
        else:
            pyboy.memory[addr] = val & 0xFF

    def write_word(addr, val):
        write_mem(addr, (val >> 8) & 0xFF)
        write_mem(addr + 1, val & 0xFF)

    def write_bcd(val):
        return ((val // 10) << 4) | (val % 10)

    print("Inyectando configuración de equipo e inventario...")

    # 1. Equipo
    team = config.get('player_team', [])
    write_mem(PARTY_SIZE_ADDRESS, len(team))
    for i, poke in enumerate(team):
        slot = poke.get('slot', 1) - 1
        if 0 <= slot < 6:
            write_mem(PARTY_ADDRESSES[slot], poke.get('species_id', 0))
            write_mem(LEVELS_ADDRESSES[slot], poke.get('level', 5))
            write_word(HP_ADDRESSES[slot], poke.get('current_hp', 20))
            write_word(MAX_HP_ADDRESSES[slot], poke.get('max_hp', 20))

    # 2. Items
    items = config.get('bag_items', [])
    item_count = min(len(items), 20)
    write_mem(BAG_ITEM_COUNT, item_count)
    for i, item in enumerate(items[:20]):
        base = BAG_ITEMS_START + (i * 2)
        write_mem(base, item.get('item_id', 0))
        write_mem(base + 1, item.get('quantity', 1))
    write_mem(BAG_ITEMS_START + (item_count * 2), 0xFF)

    # 3. Dinero y Medallas
    money = config.get('money', 0)
    write_mem(MONEY_ADDRESS_1, write_bcd(money // 10000))
    write_mem(MONEY_ADDRESS_2, write_bcd((money // 100) % 100))
    write_mem(MONEY_ADDRESS_3, write_bcd(money % 100))
    write_mem(BADGE_COUNT_ADDRESS, config.get('badge_bits', 0))

    # 4. Warp Seguro
    start_pos = config.get('start_position', {'x': 4, 'y': 13})
    map_id = config.get('map_id', 0)
    
    print(f"🌀 Programando Warp a Mapa {map_id} ({start_pos['x']}, {start_pos['y']})...")
    write_mem(0xD365, map_id)          # wWarpDestMap
    write_mem(0xD366, start_pos['x'])  # wWarpDestX
    write_mem(0xD367, start_pos['y'])  # wWarpDestY
    
    if hasattr(pyboy, "get_memory_value"):
        current_wd72d = pyboy.get_memory_value(0xD12B)
    else:
        current_wd72d = pyboy.memory[0xD12B]
    write_mem(0xD12B, current_wd72d | 0x08) # Trigger Warp
    write_mem(0xD35D, 0x00) # Reset script state

    return map_id, start_pos

def evaluate_gym_scenario(model_path, scenario_path, headless=True):
    """Ejecuta la evaluación completa de un escenario de gimnasio."""
    
    # Rutas
    state_file = os.path.join(scenario_path, "gym_scenario.state")
    config_file = os.path.join(scenario_path, "team_config.json")
    
    if not os.path.exists(model_path):
        print(f"Modelo no encontrado: {model_path}")
        return
    if not os.path.exists(state_file):
        print(f"Estado no encontrado: {state_file}")
        return
    if not os.path.exists(config_file):
        print(f"Configuración no encontrada: {config_file}")
        return

    # Cargar Configuración
    with open(config_file, 'r') as f:
        team_config = json.load(f)
        
    print(f"\nEvaluando en: {team_config.get('gym_name', 'Unknown Gym')}")
    print(f"Modelo: {os.path.basename(model_path)}")
    print(f"Modo Headless: {headless}")

    # Configurar Entorno
    env_overrides = build_env_overrides(state_file, headless=headless)
    env_overrides['max_steps'] = 2048 * 5 
    base_config = _base_env_config(env_overrides)
    
    # Instanciar Agente
    agent_config = CombatAgentConfig(env_config=base_config, total_timesteps=1000)
    agent_wrapper = CombatApexAgent(agent_config)
    
    # Cargar Modelo
    print("Cargando pesos del modelo...")
    agent_wrapper.model = PPO.load(model_path)
    
    # Crear Entorno
    env = agent_wrapper.make_env()
    
    # --- FIX: Manejo de VecEnv vs Env estándar ---
    try:
        obs, _ = env.reset()
    except ValueError:
        # Si falla, es probable que sea un VecEnv que solo retorna obs
        obs = env.reset()
    
    # Inyectar Configuración
    target_map, target_pos = inject_gym_config(env, team_config)
    base_env = get_base_env(env) # Referencia directa para lecturas de memoria
    
    # Calentamiento para Warp
    print("Calentando motor para warp (3s)...")
    for _ in range(180):
        base_env.pyboy.tick(1, False)
        if not headless:
            env.render() # Mantener ventana viva
            
    # Verificar si el Warp funcionó
    current_map = base_env.read_m(0xD35E)
    if current_map != target_map:
        print(f"⚠️ ADVERTENCIA: El Warp falló. Mapa actual: {current_map}, Esperado: {target_map}")
        print("   Reintentando inyección...")
        inject_gym_config(env, team_config)
        for _ in range(60):
            base_env.pyboy.tick(1, False)
            if not headless: env.render()
        
    # Inicializar Tracker
    tracker = GymMetricsTracker(
        gym_number=team_config.get('gym_number', 1),
        agent_name="CombatApex_Local",
        gym_name=team_config.get('gym_name', "")
    )
    tracker.start()
    
    done = False
    truncated = False
    
    print("\nIniciando ejecución del agente...")
    while not done and not truncated:
        
        # --- FIX: Asegurar que obs no sea tupla antes de predict ---
        if isinstance(obs, tuple):
            # print(f"DEBUG: Obs es tupla {type(obs)}, corrigiendo...")
            obs = obs[0]
            
        # Predecir
        action, _ = agent_wrapper.model.predict(obs, deterministic=True)
        
        # Ejecutar
        step_result = env.step(action)
        
        # --- FIX: Unpacking flexible para VecEnv ---
        if len(step_result) == 4:
            obs, reward, done, info = step_result
            truncated = False
            # Si es VecEnv, reward/done son arrays
            if isinstance(done, (list, np.ndarray)): done = done[0]
            if isinstance(reward, (list, np.ndarray)): reward = reward[0]
            if isinstance(info, (list, np.ndarray)): info = info[0]
        else:
            obs, reward, done, truncated, info = step_result
        
        # Renderizar
        if not headless:
            env.render()
            
        # Registrar métricas (Usando base_env para leer memoria)
        # FIX: Asegurar tipos nativos de Python para evitar errores de JSON serialization
        game_state = {
            'x': int(base_env.read_m(0xD362)),
            'y': int(base_env.read_m(0xD361)),
            'map': int(base_env.read_m(0xD35E)),
            'hp': [int(base_env.read_m(HP_ADDRESSES[i])) for i in range(6)],
            'in_battle': bool(base_env.read_m(0xD057) != 0)
        }
        
        # FIX: Convertir acción y reward a escalares nativos
        action_scalar = action.item() if isinstance(action, np.ndarray) else action
        reward_scalar = float(reward)
        
        tracker.record_step(action_scalar, reward_scalar, game_state)
        
        # Lógica de batalla
        if game_state['in_battle'] and not tracker.battle_started:
            tracker.record_battle_start()
        elif not game_state['in_battle'] and tracker.battle_started:
            tracker.record_battle_end(won=True) 
            
        # Seguridad
        if game_state['map'] != target_map and not game_state['in_battle']:
             pass

    env.close()
    
    # Finalizar
    tracker.end(success=tracker.battle_won)
    tracker.save_metrics(output_dir="metrics_evaluation")
    
    stats = tracker.get_summary_stats()
    print("\nResumen de Evaluación:")
    print(json.dumps(stats, indent=2))
    
    # --- DIAGNÓSTICO AUTOMÁTICO ---
    if not stats['battle_won']:
        print("\n⚠️ DIAGNÓSTICO DE FALLO:")
        print("   'FALLO' significa que el agente no ganó la batalla del gimnasio.")
        if stats['unique_tiles_explored'] <= 1:
            print("   🔴 EL AGENTE ESTÁ INMÓVIL: Solo exploró 1 baldosa.")
            print("   Sugerencia: Revisa si el modelo aprendió correctamente o si el Warp lo dejó atrapado.")
        elif stats['battle_steps'] == 0:
            print("   🟠 NO ENTRÓ A BATALLA: El agente se movió pero no inició el combate.")
            
    return stats

# --- EJECUTAR EVALUACIÓN ---
MODEL_PATH = os.path.join(project_path, 'models_local', 'combat', 'pewter_brock_battle.zip')
SCENARIO_PATH = os.path.join(project_path, 'gym_scenarios', 'gym1_pewter_brock')

# Ejecutar (Cambiado a headless=False para ver qué pasa)
stats = evaluate_gym_scenario(MODEL_PATH, SCENARIO_PATH, headless=False)


Evaluando en: Pewter City Gym - Brock
Modelo: pewter_brock_battle.zip
Modo Headless: False
Cargando pesos del modelo...
Inyectando configuración de equipo e inventario...
🌀 Programando Warp a Mapa 54 (4, 13)...
Calentando motor para warp (3s)...
Inyectando configuración de equipo e inventario...
🌀 Programando Warp a Mapa 54 (4, 13)...
Calentando motor para warp (3s)...
📊 Métricas iniciadas para CombatApex_Local en Gimnasio 1

Iniciando ejecución del agente...
📊 Métricas iniciadas para CombatApex_Local en Gimnasio 1

Iniciando ejecución del agente...


c:\Users\javi1\anaconda3\envs\pokeenv\lib\site-packages\stable_baselines3\common\vec_env\base_vec_env.py:243: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


KeyboardInterrupt: 